# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [70]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [27]:
#how many black-sounding names are in the sample
np.sum(data['race']=='b')

2435

In [28]:
#how many white-sounding names are in the sample
np.sum(data['race']=='w')

2435

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


Answer to Q1 and Q2
1. The appropriate test for this problem is a two sampled t-test and the CLT does apply because the CLT states that with independent variables; regardless of the distribution of the sample population, the distribution of the sum of the sample will tend towards normal. 
2. Null hypothesis is that the mean number of callbacks on the white-sounding names is the same as the mean number of callbacks for the black-sounding names. 

In [7]:
w = data[data.race=='w']
b = data[data.race=='b']

In [42]:
# Your solution to Q3 here
#frequentist approach
t_value, p_value = stats.ttest_ind(w.call,b.call)
print("P-Value in regards to frequentist approach is:", p_value)
std_w = np.std(w.call)
mean_w = np.mean(w.call)

conf_int = stats.norm.interval(0.95, loc=mean_w, scale = std_w/np.sqrt(len(w.call)))
print("The 95% confidence interval is:", conf_int)

print("The standard error is:", std_w)


P-Value in regards to frequentist approach is: 3.94080210313e-05
The 95% confidence interval is: (0.084780812755824247, 0.1082376683795441)
The standard error is: 0.29528486728668213


In [73]:
#bootstrap approach
def bootstrap(sample, func,n=None):
    if n == None:
        n = len(sample)
    bs_dist = np.empty(n)
    for i in range(n):
        new_sample = np.random.choice(sample, size=len(sample))
        bs_dist[i] = func(new_sample)
    return bs_dist

#calculating bootstrap distributino where w_mean = b_mean and 100000 repetitions
w_bs = bootstrap(w.call, np.mean, 100000)
w_mean = np.mean(w.call)
b_mean = np.mean(b.call)

#calculating confidence interval, margin of error and p-value based on sampling distribution
mu = np.mean(w_bs)
std = np.std(w_bs)
lower_95 = mu - 2*std
upper_95 = mu + 2*std
z_score = (b_mean - mu)/std
p_value = stats.norm.sf(abs(z_score))*2
print("The 95% confidence interval is:",(lower_95, upper_95))
print("The standard error is:", std)
print("The p-value is: ", p_value)

The 95% confidence interval is: (0.084562400079288058, 0.10848934528463286)
The standard error is: 0.00598173630134
The p-value is:  8.41993301981e-08


In [74]:
b_mean

0.0644763857126236

Answer to Q4 and Q5

A4. When comparing the mean of the call back for applicants with black names versus the mean of the number of call backs of white applicants, there is statistical significance between the two populations.  
If we are to assume that the population of white applicants is the same as that of the black applicants, then the probability of getting a mean as low as 0.064 is less than 0.01%.  This means that it is very likely the population of white names and black names are different.     

#A5. According to my analysis, it appears that race is significant in determining the call back rate.  When measuring the mean of the the black names against the population from the white names, the probability that the same mean would occur from the same population (white names) is very small (8.42e-8 and 3.94e-5).  